In [2]:
from elasticsearch import Elasticsearch
import os
import openai


In [3]:
openai.api_key = os.getenv('API_KEY')


In [4]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth = ("elastic", "ElasticSear"),
    ca_certs = "/Users/yousra/TollEtatenFinal/elasticsearch-8.12.1/config/certs/http_ca.crt"
)
es.ping()

False

## Preparing the data

In [5]:
import pandas as pd

# Read the CSV file without a header row, and specify column names manually
column_names = ['Varenummer', 'Kapittel', 'Beskrivelse']  # Example column names, adjust as needed
df = pd.read_csv("Bku.csv", sep=';', header=None, names=column_names).loc[:100]

df.head()

#import pandas as pd

# Read the CSV file without a header row, and specify column names manually
#column_names = ['Varenummer', 'Kapittel', 'Beskrivelse']  # Example column names, adjust as needed
#df = pd.read_json("tolltariffstruktur-kopi.json").loc[:100]

#df.head()


,versjon,avsnitt
0,1.5,"{'type': 'avsnitt', 'id': '01', 'beskrivelse':..."
1,1.5,"{'type': 'avsnitt', 'id': '02', 'beskrivelse':..."
2,1.5,"{'type': 'avsnitt', 'id': '03', 'beskrivelse':..."
3,1.5,"{'type': 'avsnitt', 'id': '04', 'beskrivelse':..."
4,1.5,"{'type': 'avsnitt', 'id': '05', 'beskrivelse':..."


In [23]:
df.isna().value_counts()

Varenummer  Kapittel  Beskrivelse
False       False     False          101
Name: count, dtype: int64

## Convert the relevant field to Vector using BERT model

In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2") #Hvorfor bruker vi denne modellen? huske å skrive ned til rapport.

In [6]:
df["DescriptionVector"] = df["Beskrivelse"].apply(lambda x: model.encode(x))

/Users/yousra/csvFilTollEtaten/venv/lib/python3.9/site-packages/transformers/models/mpnet/modeling_mpnet.py:1054: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:425.)
  incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask


In [7]:
df.head()

,Varenummer,Kapittel,Beskrivelse,DescriptionVector
0,10059010,Grovfor til storfe,"Hele maisplanter, med maiskolbe, som er finsni...","[0.028665954, -0.08935317, 0.010306026, 0.0625..."
1,10062090,Fullkorns basmatiris 500 gram,"Varen er ukokt, avskallet ""brown"" fullkorns ba...","[0.059810903, -0.1286022, 0.019086884, 0.06031..."
2,10062090,Ris,"Varen er avlange, avskallede rå riskorn av tha...","[0.028454714, -0.14680699, -0.010953661, 0.029..."
3,10062090,Ris,"Varen ukokt, avskallet, upolert, ""svart jasmin...","[0.07438945, -0.06828352, -0.0195118, 0.015349..."
4,10062090,Ris,"Varen langkornet, avskallet, upolert ris, som ...","[0.034095924, -0.1292016, 0.0064342725, 0.0312..."


In [25]:
es.ping()

True

## Create new index in ElasticSearch


In [26]:
from indexmapping import indexmapping
es.indices.create(index = "all_information", mappings = indexmapping)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [all_information/z18DIRwRRm6A6WBg6U6fug] already exists')

## Ingest the data into index

In [27]:
record_list = df.to_dict("records")



In [28]:
record_list[0]

{'Varenummer': 10059010,
 'Kapittel': 'Grovfor til storfe',
 'Beskrivelse': 'Hele maisplanter, med maiskolbe, som er finsnittet og ensilert/konservert uten tilsetninger i store plansiloer med plast på toppen. Dette blir lastet opp i lastebil og kjørt i bulk til Norge eller blir presset i runde baller med plast på utsiden. Brukes som grovfor til storfe.'}

In [29]:
for record in record_list:
    try:
        es.index(index = "all_information", document = record, id = record["Varenummer"])
    except Exception as e: 
        print(e)



In [30]:
es.count(index = "all_information" )

ObjectApiResponse({'count': 22, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [1]:
openai.api_key = 'API_KEY'

def utvid_sokeforesporsel(foresporsel):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Eller annen tilgjengelig chatmodell
        messages=[
            {"role": "system", "content": "Du er en svært intelligent assistent."},
            {"role": "user", "content": foresporsel}
        ]
    )
    utvidet_foresporsel = response.choices[0].message['content']
    return utvidet_foresporsel

inpuut_keyword = "mais"  # Et input som vi sender til DescriptionVector der vi har embeddings
vector_of_input_keyword = model.encode(inpuut_keyword)

# Oppdatert for å teste forskjellige verdier
k_value = 10  # Øker antall resultater som ønskes
num_candidates_value = 1000  # Øker antall kandidater som vurderes

query = {
    "field": "DescriptionVector",
    "query_vector": vector_of_input_keyword,
    "k": k_value,  # Oppdatert antall resultater som ønskes
    "num_candidates": num_candidates_value,  # Oppdatert antall kandidater som vurderes
}

res = es.knn_search(index="all_information", knn=query, source=["Varenummer", "Kapittel", "Beskrivelse"])
res["hits"]["hits"]


NameError: name 'openai' is not defined